In [4]:
from typing import List
import numpy as np

from pyquil import Program
from pyquil.gates import MEASURE, I, CNOT, X, H, CZ, RY
from pyquil.quil import address_qubits
from pyquil.quilatom import QubitPlaceholder
from pyquil.api import QVMConnection
from pyquil.noise import pauli_kraus_map, dephasing_kraus_map
from qss import NN_encode, NN_decode, NN_test


%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
qvm = QVMConnection()

In [6]:
test = QubitPlaceholder()
pq = Program()
# pq += X(XX)

## rotate into certain state
pq += RY(2.0*np.pi/3.0,test)

N_encode, N_decode = 4, 4

alpha_sqrd, beta_sqrd = NN_test(test, pq, N_encode, N_decode)

alpha squared = 0.28700000000000003
beta squared = 0.713
